In [ ]:
import pandas as pd

df = pd.read_csv("/content/train_data_with_rank.csv")

In [ ]:
df

,question,positive_passage,Rank,negative_passage
0,من هم قوم شعيب؟,وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله م...,1,NaN
1,من هم قوم شعيب؟,وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله م...,2,NaN
2,من هم قوم شعيب؟,كذب أصحاب الأيكة المرسلين. إذ قال لهم شعيب ألا...,3,NaN
3,من هم قوم شعيب؟,وإلى مدين أخاهم شعيبا فقال يا قوم اعبدوا الله ...,4,NaN
4,من هم قوم موسى؟,ثم بعثنا من بعدهم موسى بآياتنا إلى فرعون وملئه...,1,NaN
...,...,...,...,...
1299,هل هناك دليل أو أدلة على وحدة الأديان التي أنز...,يا أيها الذين آمنوا اركعوا واسجدوا واعبدوا ربك...,15,NaN
1300,هل هناك دليل أو أدلة على وحدة الأديان التي أنز...,ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن إلا ا...,16,NaN
1301,هل هناك دليل أو أدلة على وحدة الأديان التي أنز...,شرع لكم من الدين ما وصى به نوحا والذي أوحينا إ...,17,NaN
1302,هل هناك دليل أو أدلة على وحدة الأديان التي أنز...,أفأنت تسمع الصم أو تهدي العمي ومن كان في ضلال ...,18,NaN


In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder, InputExample

# 1. Load dataset
df = pd.read_csv("/content/drive/MyDrive/QuranQA/Rerank/train_data_with_rank.csv")

# 2. Keep only top-20 results per question
df = df[df["Rank"] <= 20]

# 3. Create graded labels
train_samples = [
    InputExample(
        texts=[row["question"], row["positive_passage"]],
        label=1.0 / row["Rank"]  # graded label
    )
    for _, row in df.iterrows()
]

# 4. Create DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

# 5. Train CrossEncoder
model = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    num_labels=1,   # regression output
    token=""
)

model.fit(
    train_dataloader=train_dataloader,
    epochs=3,
    output_path="/content/drive/MyDrive/QuranQA/Rerank",
    save_best_model=True
)


Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Step,Training Loss


In [ ]:
model.save("/content/drive/MyDrive/QuranQA/Rerank/quran_hadith_reranker_model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sentence_transformers import CrossEncoder

# 1. Load dataset
df = pd.read_csv("/content/drive/MyDrive/QuranQA/Rerank/train_data_with_rank.csv")
df = df[df["Rank"] <= 20]

# 2. Load trained model from Drive
model = CrossEncoder("/content/drive/MyDrive/QuranQA/Rerank/quran_hadith_reranker_model")

# 3. Select one question and its top 20 candidates
question = "من هم قوم شعيب؟"
subset = df[df["question"] == question]

# 4. Prepare pairs for prediction
pairs = [[question, passage] for passage in subset["positive_passage"]]

# 5. Get model scores
scores = model.predict(pairs)

# 6. Show results sorted by predicted score
results = list(zip(subset["positive_passage"], subset["Rank"], scores))
results_sorted = sorted(results, key=lambda x: x[2], reverse=True)

for passage, original_rank, score in results_sorted:
    print(f"Predicted score: {score:.4f} | Original Rank: {original_rank} | Passage: {passage}")


Predicted score: 7.4401 | Original Rank: 4 | Passage: وإلى مدين أخاهم شعيبا فقال يا قوم اعبدوا الله وارجوا اليوم الآخر ولا تعثوا في الأرض مفسدين. فكذبوه فأخذتهم الرجفة فأصبحوا في دارهم جاثمين.
Predicted score: 6.9655 | Original Rank: 2 | Passage: وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله ما لكم من إله غيره ولا تنقصوا المكيال والميزان إني أراكم بخير وإني أخاف عليكم عذاب يوم محيط. ويا قوم أوفوا المكيال والميزان بالقسط ولا تبخسوا الناس أشياءهم ولا تعثوا في الأرض مفسدين. بقيت الله خير لكم إن كنتم مؤمنين وما أنا عليكم بحفيظ. قالوا يا شعيب أصلاتك تأمرك أن نترك ما يعبد آباؤنا أو أن نفعل في أموالنا ما نشاء إنك لأنت الحليم الرشيد. قال يا قوم أرأيتم إن كنت على بينة من ربي ورزقني منه رزقا حسنا وما أريد أن أخالفكم إلى ما أنهاكم عنه إن أريد إلا الإصلاح ما استطعت وما توفيقي إلا بالله عليه توكلت وإليه أنيب.
Predicted score: 6.8703 | Original Rank: 1 | Passage: وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله ما لكم من إله غيره قد جاءتكم بينة من ربكم فأوفوا الكيل والميزان ولا تبخسوا الناس أشياءهم ولا ت

In [ ]:
import pandas as pd
from sentence_transformers import CrossEncoder

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/QuranQA/Rerank/train_data_with_rank.csv")
df = df[df["Rank"] <= 20]

# Load trained model
model = CrossEncoder("/content/drive/MyDrive/QuranQA/Rerank/quran_hadith_reranker_model")

def rerank_question(question_text):
    subset = df[df["question"] == question_text]
    if subset.empty:
        print("⚠️ No results found for this question.")
        return

    pairs = [[question_text, passage] for passage in subset["positive_passage"]]
    scores = model.predict(pairs)

    results = list(zip(subset["positive_passage"], subset["Rank"], scores))
    results_sorted = sorted(results, key=lambda x: x[2], reverse=True)

    for passage, original_rank, score in results_sorted:
        print(f"Predicted score: {score:.4f} | Original Rank: {original_rank} | Passage: {passage}")
    print("-" * 80)

# Try multiple questions
rerank_question("من هم قوم شعيب؟")
rerank_question("من هم قوم موسى؟")


Predicted score: 7.4401 | Original Rank: 4 | Passage: وإلى مدين أخاهم شعيبا فقال يا قوم اعبدوا الله وارجوا اليوم الآخر ولا تعثوا في الأرض مفسدين. فكذبوه فأخذتهم الرجفة فأصبحوا في دارهم جاثمين.
Predicted score: 6.9655 | Original Rank: 2 | Passage: وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله ما لكم من إله غيره ولا تنقصوا المكيال والميزان إني أراكم بخير وإني أخاف عليكم عذاب يوم محيط. ويا قوم أوفوا المكيال والميزان بالقسط ولا تبخسوا الناس أشياءهم ولا تعثوا في الأرض مفسدين. بقيت الله خير لكم إن كنتم مؤمنين وما أنا عليكم بحفيظ. قالوا يا شعيب أصلاتك تأمرك أن نترك ما يعبد آباؤنا أو أن نفعل في أموالنا ما نشاء إنك لأنت الحليم الرشيد. قال يا قوم أرأيتم إن كنت على بينة من ربي ورزقني منه رزقا حسنا وما أريد أن أخالفكم إلى ما أنهاكم عنه إن أريد إلا الإصلاح ما استطعت وما توفيقي إلا بالله عليه توكلت وإليه أنيب.
Predicted score: 6.8703 | Original Rank: 1 | Passage: وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله ما لكم من إله غيره قد جاءتكم بينة من ربكم فأوفوا الكيل والميزان ولا تبخسوا الناس أشياءهم ولا ت

In [ ]:
from sentence_transformers import CrossEncoder, InputExample, losses


In [ ]:
model = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    num_labels=1,   # regression output
    token=""
)

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder, InputExample, losses
import itertools

# 1. Load dataset
df = pd.read_csv("/content/drive/MyDrive/QuranQA/Rerank/train_data_with_rank.csv")

# 2. Keep only top-20 results per question
df = df[df["Rank"] <= 20]

# 3. Build pairwise training samples: (question, better_passage, worse_passage)
train_samples = []
for q, group in df.groupby("question"):
    group_sorted = group.sort_values("Rank")  # lower rank = better
    passages = group_sorted["positive_passage"].tolist()
    ranks = group_sorted["Rank"].tolist()

    # Make all possible better-worse pairs
    for i, j in itertools.combinations(range(len(passages)), 2):
        if ranks[i] < ranks[j]:  # i is better than j
            train_samples.append(InputExample(
                texts=[q, passages[i], passages[j]],  # better, worse
                label=1  # margin ranking target
            ))
        elif ranks[i] > ranks[j]:  # j is better than i
            train_samples.append(InputExample(
                texts=[q, passages[j], passages[i]],  # better, worse
                label=1
            ))

# 4. DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=8)

# 5. Model
model = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    num_labels=1,   # regression output
    token=""
)

# 6. Loss function for pairwise ranking
train_loss = losses.MarginRankingLoss(model=model)

# 7. Train
model.fit(
    train_dataloader=train_dataloader,
    loss_fct=train_loss,
    epochs=5,  # train longer than before
    warmup_steps=int(0.1 * len(train_dataloader) * 5),
    output_path="/content/drive/MyDrive/QuranQA/Rerank_pairwise",
    optimizer_params={'lr': 2e-5},
    save_best_model=True
)


AttributeError: module 'sentence_transformers.losses' has no attribute 'MarginRankingLoss'

In [ ]:
claude_df

,"question id,answe claude"
0,"653,﴿إِنَّ فِي خَلْقِ السَّمَاوَاتِ وَالْأَرْض..."
1,"653,﴿وَفِي الْأَرْضِ آيَاتٌ لِّلْمُوقِنِينَ﴾ [..."
2,"653,﴿سَنُرِيهِمْ آيَاتِنَا فِي الْآفَاقِ وَفِي..."
3,"736,﴿وَمَن يَعْمَلْ سُوءًا أَوْ يَظْلِمْ نَفْس..."
4,"736,﴿وَالَّذِينَ إِذَا فَعَلُوا فَاحِشَةً أَوْ..."
...,...
120,"670,﴿وَهُوَ الَّذِي مَرَجَ الْبَحْرَيْنِ هَٰذَ..."
121,"670,﴿مَرَجَ الْبَحْرَيْنِ يَلْتَقِيَانِ * بَيْ..."
122,"700,﴿وَمَا تُقَدِّمُوا لِأَنفُسِكُم مِّنْ خَيْ..."
123,"700,﴿إِنَّمَا نُطْعِمُكُمْ لِوَجْهِ اللَّهِ لَ..."


In [ ]:
claude_df.columns

Index(['question id,answe claude'], dtype='object')

In [ ]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# ===== 1. Load your data =====
claude_df = pd.read_csv("/content/claude-test_last - Sheet1.csv", sep=",")  # change sep if CSV
quran_df = pd.read_csv("/content/QH-QA-25_Subtask2_QPC_v1.1 (1).tsv", sep="\t", header=None, names=["verse_id", "verse_text"])

# ===== 2. Text cleaning =====
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove diacritics and non-Arabic chars except spaces
    text = re.sub(r"[ًٌٍَُِّْـ]", "", text)  # remove tashkeel
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  # remove non-Arabic except space
    return re.sub(r"\s+", " ", text).strip()

claude_df["cleaned_answer"] = claude_df["answer claude"].apply(clean_text)
quran_df["cleaned_text"] = quran_df["verse_text"].apply(clean_text)

# ===== 3. Match each Claude excerpt to Qur’an verse =====
matches = []
for idx, row in claude_df.iterrows():
    query = row["cleaned_answer"]
    best_match = process.extractOne(query, quran_df["cleaned_text"], scorer=fuzz.partial_ratio)
    if best_match:
        match_text = quran_df.iloc[best_match[2]]["verse_text"]
        match_id = quran_df.iloc[best_match[2]]["verse_id"]
        matches.append((row["question id"], row["answer claude"], match_text, match_id))
    else:
        matches.append((row["question id"], row["answer claude"], "-1", "-1"))

# ===== 4. Save output =====
result_df = pd.DataFrame(matches, columns=["question id", "answer claude", "quran passage", "quran passage id"])
result_df.to_csv("claude_quran_matched_dev.csv", index=False)


In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import ast
from rapidfuzz import process, fuzz

# === 1. Load the matched Quran file ===
df = pd.read_csv("/content/claude_quran_matched_test.csv")

# === 2. Load Sahih Bukhari JSONL-like file using ast.literal_eval ===
hadith_records = []
with open("/content/QH-QA-25_Subtask2_Sahih-Bukhari_v1.0.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = ast.literal_eval(line)
            hadith_records.append(record)
        except Exception as e:
            print(f"Skipping invalid line: {line}\nError: {e}")

hadith_df = pd.DataFrame(hadith_records)

# === 3. Clean text function ===
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove Arabic diacritics (tashkeel)
    text = re.sub(r"[ًٌٍَُِّْـ]", "", text)
    # Keep only Arabic letters and spaces
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    # Normalize spaces
    return re.sub(r"\s+", " ", text).strip()

# === 4. Clean Claude answers and Hadith texts ===
df["cleaned_answer"] = df["answer claude"].apply(clean_text)
hadith_df["cleaned_hadith"] = hadith_df["hadith"].apply(clean_text)

# === 5. Match every Claude answer to the best Hadith ===
hadith_matches = []
for idx, row in df.iterrows():
    query = row["cleaned_answer"]
    best_match = process.extractOne(query, hadith_df["cleaned_hadith"], scorer=fuzz.partial_ratio)
    if best_match:
        match_row = hadith_df.iloc[best_match[2]]
        hadith_matches.append((match_row["hadith"], match_row["hadith_id"]))
    else:
        hadith_matches.append(("-1", "-1"))

# === 6. Add Hadith columns to dataframe ===
df["hadith"] = [m[0] for m in hadith_matches]
df["hadith id"] = [m[1] for m in hadith_matches]

# === 7. Save result ===
df.to_csv("claude_quran_hadith_matched_test_final.csv", index=False)
print("Saved claude_quran_hadith_matched.csv successfully.")


Saved claude_quran_hadith_matched.csv successfully.


In [ ]:
import pandas as pd

input_csv = "/content/submit/merged_quran_hadith - merged_quran_hadith.csv"
output_tsv = "submission_test_codabench.tsv"
tag = "ayatec_v2.1_test_hybrid"

df = pd.read_csv(input_csv)

results = []
for idx, row in df.iterrows():
    qid = str(row["question id"])
    pid = str(row["passage id"]) if pd.notna(row["passage id"]) and row["passage id"] != "" else "-1"
    results.append([qid, "0", pid, 1, 1, tag])

out_df = pd.DataFrame(results, columns=["question-id", "0", "passage-id", "rank", "score", "tag"])
out_df.to_csv(output_tsv, sep="\t", index=False, header=False)
print(f"Saved submission to {output_tsv}")


Saved submission to submission_test_codabench.tsv


In [ ]:
out_df

,question-id,0,passage-id,rank,score,tag
0,653,0,3:190-195,1,1,ayatec_v2.1_test_hybrid
1,653,0,51:15-23,1,1,ayatec_v2.1_test_hybrid
2,653,0,41:53-54,1,1,ayatec_v2.1_test_hybrid
3,653,0,42:27-31,1,1,ayatec_v2.1_test_hybrid
4,736,0,3:133-136,1,1,ayatec_v2.1_test_hybrid
...,...,...,...,...,...,...
224,700,0,1,1,1,ayatec_v2.1_test_hybrid
225,700,0,18:109-110,1,1,ayatec_v2.1_test_hybrid
226,700,0,98:1-5,1,1,ayatec_v2.1_test_hybrid
227,743,0,33:32-34,1,1,ayatec_v2.1_test_hybrid


In [ ]:
out_df = out_df.drop_duplicates(subset=["question-id", "passage-id"])
out_df

,question-id,0,passage-id,rank,score,tag
0,653,0,3:190-195,1,1,ayatec_v2.1_test_hybrid
1,653,0,51:15-23,1,1,ayatec_v2.1_test_hybrid
2,653,0,41:53-54,1,1,ayatec_v2.1_test_hybrid
3,653,0,42:27-31,1,1,ayatec_v2.1_test_hybrid
4,736,0,3:133-136,1,1,ayatec_v2.1_test_hybrid
...,...,...,...,...,...,...
224,700,0,1,1,1,ayatec_v2.1_test_hybrid
225,700,0,18:109-110,1,1,ayatec_v2.1_test_hybrid
226,700,0,98:1-5,1,1,ayatec_v2.1_test_hybrid
227,743,0,33:32-34,1,1,ayatec_v2.1_test_hybrid


In [ ]:
out_df.to_csv("final_submission_codabench_final.tsv", sep="\t", index=False, header=False)
print(f"Saved deduplicated submission to {output_tsv}")

Saved deduplicated submission to submission_test_codabench.tsv


In [ ]:
import pandas as pd
import re

# ==== Load Data ====
questions_df = pd.read_csv("/content/keyword_test - Sheet1.csv")  # has: question id, keywords
quran_df = pd.read_csv("/content/QH-QA-25_Subtask2_QPC_v1.1 (1).tsv", sep="\t", header=None, names=["verse_id", "verse_text"])
hadith_df = pd.read_json("/content/QH-QA-25_Subtask2_Sahih-Bukhari_v1.0.jsonl", lines=True)

# ==== Text cleaning ====
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"[ًٌٍَُِّْـ]", "", text)   # remove diacritics
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  # keep only Arabic + space
    return re.sub(r"\s+", " ", text).strip()

quran_df["cleaned"] = quran_df["verse_text"].apply(clean_text)
hadith_df["cleaned"] = hadith_df["hadith"].apply(clean_text)

# ==== Search for all matches ====
results = []

for _, row in questions_df.iterrows():
    qid = row["question id"]
    keywords = [clean_text(k) for k in str(row["keywords"]).split(",")]

    # Qur’an matches
    for kw in keywords:
        if not kw.strip():
            continue
        quran_matches = quran_df[quran_df["cleaned"].str.contains(kw)]
        for _, qm in quran_matches.iterrows():
            results.append({
                "question id": qid,
                "source": "quran",
                "keyword": kw,
                "passage": qm["verse_text"],
                "passage id": qm["verse_id"]
            })

        # Hadith matches
        hadith_matches = hadith_df[hadith_df["cleaned"].str.contains(kw)]
        for _, hm in hadith_matches.iterrows():
            results.append({
                "question id": qid,
                "source": "hadith",
                "keyword": kw,
                "passage": hm["hadith"],
                "passage id": hm["hadith_id"]
            })

# ==== Save output ====
results_df = pd.DataFrame(results).drop_duplicates()
results_df.to_csv("all_matches__keyword_quran_hadith.csv", index=False, encoding="utf-8-sig")

print(f"✅ Saved {len(results_df)} matches to all_matches_quran_hadith.csv")


ValueError: Expected object or value

In [ ]:
import pandas as pd
import re
import ast

# ==== 1. Load Keywords ====
questions_df = pd.read_csv("/content/keyword_test - Sheet1.csv")  # Columns: question id, keywords

# ==== 2. Load Qur’an ====
quran_df = pd.read_csv(
    "/content/QH-QA-25_Subtask2_QPC_v1.1 (1).tsv",
    sep="\t",
    header=None,
    names=["verse_id", "verse_text"]
)

# ==== 3. Load Sahih Bukhari (Python dict string format) ====
hadith_records = []
with open("/content/QH-QA-25_Subtask2_Sahih-Bukhari_v1.0.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = ast.literal_eval(line)  # safely convert to dict
            hadith_records.append(record)
        except Exception as e:
            print(f"Skipping invalid line: {line[:100]}...\nError: {e}")

hadith_df = pd.DataFrame(hadith_records)

# ==== 4. Text cleaning ====
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove Arabic diacritics
    text = re.sub(r"[ًٌٍَُِّْـ]", "", text)
    # Keep only Arabic letters and spaces
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    # Normalize spaces
    return re.sub(r"\s+", " ", text).strip()

quran_df["cleaned"] = quran_df["verse_text"].apply(clean_text)
hadith_df["cleaned"] = hadith_df["hadith"].apply(clean_text)

# ==== 5. Search all matches ====
results = []

for _, row in questions_df.iterrows():
    qid = row["question id"]
    # Split keywords, clean each
    keywords = [clean_text(k) for k in str(row["keywords"]).split(",")]

    for kw in keywords:
        if not kw.strip():
            continue

        # Qur’an matches
        quran_matches = quran_df[quran_df["cleaned"].str.contains(kw)]
        for _, qm in quran_matches.iterrows():
            results.append({
                "question id": qid,
                "source": "quran",
                "keyword": kw,
                "passage": qm["verse_text"],
                "passage id": qm["verse_id"]
            })

        # Hadith matches
        hadith_matches = hadith_df[hadith_df["cleaned"].str.contains(kw)]
        for _, hm in hadith_matches.iterrows():
            results.append({
                "question id": qid,
                "source": "hadith",
                "keyword": kw,
                "passage": hm["hadith"],
                "passage id": hm["hadith_id"]
            })

# ==== 6. Save output ====
results_df = pd.DataFrame(results).drop_duplicates()
results_df.to_csv("all_matches_quran_hadith.csv", index=False, encoding="utf-8-sig")

print(f"✅ Saved {len(results_df)} matches to all_matches_quran_hadith.csv")


✅ Saved 9 matches to all_matches_quran_hadith.csv


In [ ]:
import pandas as pd
import re
import ast
from rapidfuzz import fuzz

# ==== 1. Load Keywords ====
questions_df = pd.read_csv("/content/keyword_test - Sheet1.csv")

# ==== 2. Load Qur’an ====
quran_df = pd.read_csv(
    "/content/QH-QA-25_Subtask2_QPC_v1.1 (1).tsv",
    sep="\t",
    header=None,
    names=["verse_id", "verse_text"]
)

# ==== 3. Load Sahih Bukhari (Python dict string format) ====
hadith_records = []
with open("/content/QH-QA-25_Subtask2_Sahih-Bukhari_v1.0.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = ast.literal_eval(line)
            hadith_records.append(record)
        except:
            pass

hadith_df = pd.DataFrame(hadith_records)

# ==== 4. Clean text ====
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"[ًٌٍَُِّْـ]", "", text)
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()

quran_df["cleaned"] = quran_df["verse_text"].apply(clean_text)
hadith_df["cleaned"] = hadith_df["hadith"].apply(clean_text)

# ==== 5. Match with fuzzy search ====
results = []

THRESHOLD = 70  # similarity score threshold

for _, row in questions_df.iterrows():
    qid = row["question id"]
    keywords = [clean_text(k) for k in str(row["keywords"]).split(",")]

    for kw in keywords:
        if not kw.strip():
            continue

        # Qur’an matches
        for _, qm in quran_df.iterrows():
            score = fuzz.partial_ratio(kw, qm["cleaned"])
            if score >= THRESHOLD:
                results.append({
                    "question id": qid,
                    "source": "quran",
                    "keyword": kw,
                    "passage": qm["verse_text"],
                    "passage id": qm["verse_id"],
                    "score": score
                })

        # Hadith matches
        for _, hm in hadith_df.iterrows():
            score = fuzz.partial_ratio(kw, hm["cleaned"])
            if score >= THRESHOLD:
                results.append({
                    "question id": qid,
                    "source": "hadith",
                    "keyword": kw,
                    "passage": hm["hadith"],
                    "passage id": hm["hadith_id"],
                    "score": score
                })

# ==== 6. Save results ====
results_df = pd.DataFrame(results).drop_duplicates()
results_df.to_csv("all_matches_quran_hadith_fuzzy.csv", index=False, encoding="utf-8-sig")

print(f"✅ Saved {len(results_df)} fuzzy matches to all_matches_quran_hadith_fuzzy.csv")


✅ Saved 380 fuzzy matches to all_matches_quran_hadith_fuzzy.csv


In [ ]:
import pandas as pd

# Load CSVs
file_main = "/content/submit/claude_quran_hadith_matched_test_final - claude_quran_hadith_matched_test_final.csv"
file_extra = "/content/submit/all_matches_quran_hadith_fuzzy_final - all_matches_quran_hadith_fuzzy.csv"

df_main = pd.read_csv(file_main)
df_extra = pd.read_csv(file_extra)

# Prepare extra file to match columns with main file
df_extra_renamed = df_extra.rename(columns={
    'source': 'answer claude',
    'passage': 'passage',
    'passage id': 'passage id'
})

# Keep only required columns from extra
df_extra_renamed = df_extra_renamed[['question id', 'answer claude', 'passage', 'passage id']]

# Merge: keep main order first, then append matching extra rows
merged = pd.concat([df_main, df_extra_renamed], ignore_index=True)

# Remove duplicates based on (question id, passage id)
merged = merged.drop_duplicates(subset=['question id', 'passage id'], keep='first')

# Keep same question order from main file
question_order = df_main['question id'].tolist()
merged['order'] = merged['question id'].apply(lambda x: question_order.index(x) if x in question_order else float('inf'))
merged = merged.sort_values(by=['order']).drop(columns=['order'])

# Save result
merged.to_csv("merged_quran_hadith.csv", index=False)

print("✅ Merged file saved as merged_quran_hadith.csv")


✅ Merged file saved as merged_quran_hadith.csv


In [ ]:
/content/submit/ayatec_v4.0_hybrid_system_test_test_final_Q_with_no_answer.tsv

In [ ]:
import pandas as pd

# ===== Load files =====
merged = pd.read_csv("/content/submit/merged_quran_hadith - merged_quran_hadith.csv")  # from Claude + keyword merge
embedding_file = "/content/submit/ayatec_v4.0_hybrid_system_test_test_final_Q_with_no_answer.tsv" # embedding run submission
emb_df = pd.read_csv(embedding_file, sep="\t", header=None)
emb_df.columns = ["question id", "col1", "passage id", "rank", "score", "system"]

# ===== Prepare question IDs to skip =====
skip_questions = set(merged.loc[merged["passage id"].astype(str) == "-1", "question id"])

# ===== Create Claude+keyword in run submission format =====
# We'll keep col1=0, rank starting from 1, score=1.0, system="keyword_claude"
claude_run_format = []
for qid in merged["question id"].unique():
    q_rows = merged[merged["question id"] == qid]
    rank = 1
    for _, row in q_rows.iterrows():
        claude_run_format.append([
            qid,
            0,
            row["passage id"],
            rank,
            1.0,  # since this is from Claude/keyword, we can mark score=1.0 or any placeholder
            "keyword_prompt"
        ])
        rank += 1

claude_df = pd.DataFrame(claude_run_format,
    columns=["question id", "col1", "passage id", "rank", "score", "system"])

# ===== Filter embeddings =====
# Skip questions that have -1 in merged file
emb_filtered = emb_df[~emb_df["question id"].isin(skip_questions)]

# Avoid duplicates: remove embedding rows whose (qid, passage id) is already in Claude list
existing_pairs = set(zip(claude_df["question id"], claude_df["passage id"].astype(str)))
emb_filtered = emb_filtered[~emb_filtered.apply(
    lambda x: (x["question id"], str(x["passage id"])) in existing_pairs,
    axis=1
)]

# ===== Combine =====
final_df = pd.concat([claude_df, emb_filtered], ignore_index=True)

# ===== Save =====
final_df.to_csv("/content/submit/final_run_submission.tsv", sep="\t", index=False, header=False)

print("✅ Saved final_run_submission.csv in run submission format")


✅ Saved final_run_submission.csv in run submission format


In [ ]:
import pandas as pd

# ===== Load files =====
merged = pd.read_csv("/content/submit/merged_quran_hadith - merged_quran_hadith.csv")  # Claude+keyword merged
embedding_file = "/content/submit/ayatec_v4.0_hybrid_system_test_test_final_Q_with_no_answer.tsv"  # run submission file
emb_df = pd.read_csv(embedding_file, sep="\t", header=None)
emb_df.columns = ["question id", "col1", "passage id", "rank", "score", "system"]

# ===== Find questions to skip embeddings for =====
skip_questions = set(merged.loc[merged["passage id"].astype(str) == "-1", "question id"])

# ===== Convert Claude+keyword to run submission format =====
claude_run_format = []
for qid in merged["question id"].unique():
    q_rows = merged[merged["question id"] == qid]
    rank = 1
    for _, row in q_rows.iterrows():
        claude_run_format.append([
            qid,
            0,
            row["passage id"],
            rank,
            1.0,  # score placeholder
            "keyword_claude"
        ])
        rank += 1

claude_df = pd.DataFrame(claude_run_format,
    columns=["question id", "col1", "passage id", "rank", "score", "system"])

# ===== Filter embeddings =====
# Skip questions with -1
emb_filtered = emb_df[~emb_df["question id"].isin(skip_questions)]

# Remove duplicates with Claude
existing_pairs = set(zip(claude_df["question id"], claude_df["passage id"].astype(str)))
emb_filtered = emb_filtered[~emb_filtered.apply(
    lambda x: (x["question id"], str(x["passage id"])) in existing_pairs,
    axis=1
)]

# ===== Merge Claude+keyword first, then embeddings =====
merged_all = pd.concat([claude_df, emb_filtered], ignore_index=True)

# ===== Keep only top 20 per question =====
final_list = []
for qid in merged_all["question id"].unique():
    q_subset = merged_all[merged_all["question id"] == qid].copy()
    q_subset = q_subset.sort_values(by=["system", "rank"], ascending=[True, True])
    q_subset["rank"] = range(1, len(q_subset) + 1)  # reassign ranks
    q_subset = q_subset.head(20)  # keep first 20
    final_list.append(q_subset)

final_df = pd.concat(final_list, ignore_index=True)

# ===== Save in run submission format =====
final_df.to_csv("final_run_submission_20_v2.tsv", sep="\t", index=False, header=False)

print("✅ Saved final_run_submission_20.csv with max 20 results per question")


✅ Saved final_run_submission_20.csv with max 20 results per question


In [ ]:
import pandas as pd

# ===== Load files =====
merged = pd.read_csv("/content/submit/merged_quran_hadith - merged_quran_hadith.csv")  # Claude+keyword merged
embedding_file = "/content/submit/ayatec_v4.0_hybrid_system_test_test_final_Q_with_no_answer.tsv"  # run submission file
emb_df = pd.read_csv(embedding_file, sep="\t", header=None)
emb_df.columns = ["question id", "col1", "passage id", "rank", "score", "system"]

# ===== Find questions to skip embeddings for =====
skip_questions = set(merged.loc[merged["passage id"].astype(str) == "-1", "question id"])

# ===== Convert Claude+keyword to run submission format =====
claude_run_format = []
for qid in merged["question id"].unique():
    q_rows = merged[merged["question id"] == qid]
    rank = 1
    for _, row in q_rows.iterrows():
        claude_run_format.append([
            qid,
            0,
            row["passage id"],
            rank,
            1.0,  # score placeholder
            "keyword_claude"
        ])
        rank += 1

claude_df = pd.DataFrame(claude_run_format,
    columns=["question id", "col1", "passage id", "rank", "score", "system"])

# ===== Filter embeddings =====
# Skip questions with -1 in merged
emb_filtered = emb_df[~emb_df["question id"].isin(skip_questions)]

# Remove duplicates already in Claude
existing_pairs = set(zip(claude_df["question id"], claude_df["passage id"].astype(str)))
emb_filtered = emb_filtered[~emb_filtered.apply(
    lambda x: (x["question id"], str(x["passage id"])) in existing_pairs,
    axis=1
)]

# ===== Merge Claude first, then embeddings =====
merged_all = pd.concat([claude_df, emb_filtered], ignore_index=True)

# ===== Remove rows where passage id = -1 AND system = ayatec_v4.0_hybrid_system_test =====
merged_all = merged_all[~((merged_all["passage id"].astype(str) == "-1") &
                          (merged_all["system"] == "ayatec_v4.0_hybrid_system_test"))]

# ===== Keep only top 20 per question =====
final_list = []
for qid in merged_all["question id"].unique():
    q_subset = merged_all[merged_all["question id"] == qid].copy()
    q_subset = q_subset.sort_values(by=["system", "rank"], ascending=[True, True])
    q_subset["rank"] = range(1, len(q_subset) + 1)  # reassign ranks
    q_subset = q_subset.head(20)  # keep first 20
    final_list.append(q_subset)

final_df = pd.concat(final_list, ignore_index=True)

# ===== Save in run submission format =====
final_df.to_csv("final_run_submission_20_filtered_v3.tsv", sep="\t", index=False, header=False)

print("✅ Saved final_run_submission_20_filtered.csv without -1 from ayatec_v4.0_hybrid_system_test")


✅ Saved final_run_submission_20_filtered.csv without -1 from ayatec_v4.0_hybrid_system_test
